In [2]:
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, pair
import random
from Crypto.Random import random
import time
#mul == g**scalar
#sum = g + g2
#div = g/g2 = g-g2

"""
KeyGen requires:
    g : generator of group
    q: length of message list
"""
def keygen(g:G1,q:int) -> tuple:

    #list comprehension version
    
     #calculate zi element <-$ Zp
    z=[random.randint(0,q) for i in range(0,q)]

    #calculate hi as g^{zi}
    hi = [g**z[i] for i in range(len(z))]
    
    #calculate hij as g^{zi*zj}
    h_ij = [[g**(z[i]*z[j]) if i!=j else 0 for j in range(len(z))] for i in range(len(z))]
    
    return hi, h_ij, z

dim = 100

group = PairingGroup('SS512')
g = group.random(G1)

    
messages = []

for i in range(0,dim):
    messages.append(random.randint(0,1000000))

hi,h_ij,z = keygen(g, q = dim)


In [13]:
"""
Commit requires:
    messages: list of messages to commit
    hi: public parameter as g^{zi} for 1<=i<=q
"""
def commit(messages:list, hi:list) -> G1:
    #default value
    C = hi[0]**(0)
    for i in range(len(messages)):
        #calculate C as hi^{mi} * .. * hn^{mn}
        C = C+(hi[i]**(messages[i]))
    return C

C = commit(messages,hi)
print(C)

[5208826865936317115128702029894626781274399125006369238919025896360992220225953418656049885089184321537355604247408971473611041893781387131303307067117264, 999680690270194636233718660668431834399956257518684260389574494823226749015949562903996668292822593568084305699410616133689890872985106360884456377267716]


In [16]:
"""
Open requires:
    i : index of message to open
    messages: list of messages
    h_ij: public parameter as g^{zi*zj}
    g: generator of group

"""
def open(i:int, messages:list, h_ij:list, g:G1) -> G1:
    #default value
    A = g**(0)
    for j in range(len(messages)):
        #skip i == j
        if(i==j):
            continue
        #calculate A as hij^{mij} * ... * hij^{mij}
        A = A+(h_ij[i][j]**(messages[j]))
    return A
A = open(0,messages,h_ij,g)
print(A)

[5432864398166794137042042580980896092418013794501959077963680261221480333162600817191267979050301276922481829004243283770265613154178702661484640140469272, 3198399711579633083546488276595845197865041522324772771462701505922712564719023384751033879612697407020646183353159369282957145396698201662767322506365764]


In [17]:
"""
Open2 is an alternative opening method which requires:
    messages: list of messages
    i : index of message to open
    hi: public parameter as g^{zi}
    z: public parameter as z1,...zq <-$ Zp
    g: generator of group
"""
def open2(messages:list,i:int,hi:list,z:list,g:G1) -> G1:
    #default value
    A = g**(0)
    for j in range(len(messages)):
        #skip i == j
        if(i==j):
            continue
        #calculate A as hi^{mj*zj}
        A = A+(hi[j]**(messages[j]*z[j]))
    return A

A2 = open2(messages,0,hi,z,g)
print(A2)

[1429097779083948394864000385815473339259971578124179051257563224693417690371370372461156203549447211995888075513365250569056211156510754512141707511481150, 6353264264459332244529220089621970524751129910130326649676093373185475207471793724994157416079662849405435880893932875403728386123325220980724672512282898]


In [9]:
"""
Verify requires:
    C: commitment value
    message: message to verify
    i: index of message in messages list
    A: proof of message at pos i
    g: generator of group
    hi: public parameter as g^{zi}
"""
def verify(C: G1, message:int, i:int, A:G1, g:G1, hi:list) -> bool:
    
    #calculate external divisor as hi^{mi}
    externsub = hi[i]**(message)
    #calculate first member of the first pairing as C/hi^{mi}
    C = C/(externsub)
    
    #calculate first pairing as e(C/hi^{mi},hi)
    t1 = pair(C,hi[i])
    #calculate second pairing as e(Ai,g)
    t2 = pair(A,g)
    return t1 == t2

print("Verify output: " + str(verify(C,messages[0],0,A,g,hi)))

Verify output: True


In [10]:
"""
Update commitment requires:
    C: commitment value
    oldmessage: the old message 
    newmessage: the new message to commit
    i: index of previous and new message
    hi: public parameter as g^{zi}
"""
def update(C:G1, oldmessage:int, newmessage:int,i:int,hi:list)-> G1:
    #calculate the new commitment C' as C*hi^{m'-m}
    newc = C+hi[i]**(newmessage-oldmessage)
    return newc
newc = update(C,messages[0],10000002,0,hi)
verified2 = verify(newc,10000002,0,A,g,hi)
if(verified2):
    print("Verifying update worked")
else:
    print("Verifying update not worked")

Verifying update worked


In [11]:
"""
Update proof requires:
    A: old proof
    newmessage: new message to commit
    oldmessage: old message
    i: index of old message
    j: index of new message
    h_ij: public parameter as g^{zi*zj}
"""
def updateProof(A:G1,newmessage:int,oldmessage:int,i:int,j:int,h_ij:list) -> G1:
    #calculate newproof Aj' as Aj*(hij^{m'-m})
    newproof = A+(h_ij[j][i]**(newmessage - oldmessage))
    return newproof

oldproof = open(1,messages,h_ij,g)
newproof = updateProof(oldproof,10000002,messages[0],0,1,h_ij)

verified3 = verify(newc,messages[1],1,newproof,g,hi)

if(verified3):
    print("Verifying update proof worked")
else:
    print("Verifying update proof not worked")

Verifying update proof worked
